In [ ]:
from __future__ import annotations

from sumpy.expansion.local import VolumeTaylorLocalExpansion
from sumpy.expansion.multipole import (
    LaplaceConformingVolumeTaylorMultipoleExpansion,
    LinearPDEConformingVolumeTaylorMultipoleExpansion,
    VolumeTaylorMultipoleExpansion,
)
from sumpy.kernel import HelmholtzKernel, LaplaceKernel, YukawaKernel  # noqa: F401
from sumpy.symbolic import make_sym_vector


order = 2
dim = 2

if 0:
    knl = LaplaceKernel(dim)
    extra_kernel_kwargs = {}
    mpole_expn_reduced_class = LaplaceConformingVolumeTaylorMultipoleExpansion

else:
    helm_k = 1.2
    knl = HelmholtzKernel(dim)
    extra_kernel_kwargs = {"k": helm_k}
    mpole_expn_reduced_class = LinearPDEConformingVolumeTaylorMultipoleExpansion

mpole_expn_reduced = mpole_expn_reduced_class(knl, order)
mpole_expn = VolumeTaylorMultipoleExpansion(knl, order)
local_expn = VolumeTaylorLocalExpansion(knl, order)

In [ ]:
reduced_wrangler = mpole_expn_reduced.expansion_terms_wrangler
full_wrangler = mpole_expn.expansion_terms_wrangler

reduced_derivatives = list(
    make_sym_vector("deriv", len(reduced_wrangler.stored_identifiers))
)
full_derivatives = reduced_wrangler.get_full_kernel_derivatives_from_stored(
    reduced_derivatives, 1
)

print(reduced_derivatives)
print(full_derivatives)

In [ ]:
full_coeffs = list(
    make_sym_vector("coeff", len(reduced_wrangler.get_full_coefficient_identifiers()))
)

reduced_coeffs = reduced_wrangler.get_stored_mpole_coefficients_from_full(
    full_mpole_coefficients=full_coeffs, rscale=1
)

print(full_coeffs)
print(reduced_coeffs)

In [ ]:
dvec = make_sym_vector("d", dim)
translated_reduce_coeffs = mpole_expn_reduced.translate_from(
    mpole_expn_reduced, reduced_coeffs, 1, dvec, 1
)
translated_full_coeffs = mpole_expn.translate_from(mpole_expn, full_coeffs, 1, dvec, 1)
translated_full_coeffs

In [ ]:
eval_reduced = sum(a * b for a, b in zip(translated_reduce_coeffs, reduced_derivatives,
                                         strict=True))
eval_full = sum(a * b for a, b in zip(translated_full_coeffs, full_derivatives,
                                      strict=True))

(eval_full - eval_reduced).simplify()